In [1]:
%load_ext autoreload
%autoreload 2

import yaml
import mesa
from mesa.experimental import JupyterViz
import solara
from matplotlib.figure import Figure
from trans_infra.trans_infra.model import TransInfraNetworkModel
import networkx as nx

In [2]:
def make_trans_graph(model):
    # TODO: update edge and node weight iteratively in model
    G_trans = model.G_trans
    fig = Figure()
    ax = fig.subplots()
    
    osmid_2_idx = {k: v for (k, v) in zip(G_trans.nodes, range(len(G_trans.nodes))) }
    
    nodes, edges = G_trans.nodes(data=True), G_trans.edges(data=True)
    
    max_road_len = max([ d['length'] for (u, v, d) in edges ])
    layout = { n[0] : [ n[1]['x'], n[1]['y'] ] for n in G_trans.nodes(data=True) }
    
    node_color = [ v['color'] for (k, v) in nodes ]
    edge_color = [ d['color'] for (u, v, d) in edges ]      
    edge_weight = [ d['length'] / max_road_len for (u, v, d) in edges ]
    node_weight = [1] * len(nodes)
    for a in model.space.get_all_cell_contents():
        node_weight[ osmid_2_idx[a.pos] ] += 3

    nx.draw_networkx(G_trans, pos=layout,
                    edge_color=edge_color, 
                    node_color=node_color,
                    width=edge_weight,
                    node_size=node_weight,
                    with_labels=False,
                    ax=ax)
    
    ax.set_title("Places and Their Connections")
    solara.FigureMatplotlib(fig)

In [3]:
def make_social_graph(model):
    fig = Figure()
    ax = fig.subplots()
    
    G_socia = nx.from_scipy_sparse_array(model.A_social)
    pos = nx.spring_layout(G_socia)
    betCent = nx.betweenness_centrality(G_socia, normalized=True, endpoints=True)
    node_color = [20000.0 * G_socia.degree(v) for v in G_socia]
    node_size =  [v * 1000 for v in betCent.values()]
    nx.draw_networkx(G_socia, pos=pos, with_labels=True,
                    node_color=node_color, node_size=node_size,
                    width=[e[2] for e in G_socia.edges.data('weight')],
                    font_size=5, ax=ax)
    
    ax.set_title("Social Ties")
    solara.FigureMatplotlib(fig)

In [4]:
def agent_portrayal(graph):
    # TODO: display per agent needs and counts
    nodes, edges = graph.nodes(data=True), graph.edges(data=True)
    
    max_road_len = max([ d['length'] for (u, v, d) in edges ])

    node_color = [ v['color'] for (k, v) in nodes ]
    edge_color = [ d['color'] for (u, v, d) in edges ]      
    edge_weight = [ d['length'] / max_road_len for (u, v, d) in edges ]
    
    return {
        "width": edge_weight,
        "edge_color": edge_color,
        "node_color": node_color,
        "node_size": 10
    }

In [5]:
osm_file = "./east_yaml.osm"

model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 10,
        "max": 100,
        "step": 1,
    },
    "graphfile": osm_file
}

In [6]:
# TODO: make for west campus & other cities
page = JupyterViz(
    TransInfraNetworkModel,
    model_params,
    measures=[make_trans_graph, make_social_graph],
    name="Transit Network Model",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page

/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/mesa/time.py:80: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)


Cannot show ipywidgets in text